# Import

In [42]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

import itertools
import math


In [43]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(1422) # Seed 고정

In [44]:
train_df = pd.read_csv('./data/train.csv')

In [45]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

# train 칼만 + 각종 피쳐 생성

In [46]:
train_x.drop(["X_10","X_11"], axis =1 ,inplace =True)

basic_col = train_x.columns

#for i in range(2,44):
for i in tqdm(range(len(train_x.columns))):
    current=0
    sum_c=[]
    z = train_x.loc[:, train_x.columns[i]]
    a = []           #필터링 된 피쳐(after)
    b = []           #필터링 전 피쳐(before)
    my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
    my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
    my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
    my_filter.H = np.array([[1.,0.]])    # Measurement function
    my_filter.P *= 1000.                 # covariance matrix
    my_filter.R = 5                      # state uncertainty
    my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.1) # process uncertainty   
    for k in z.values:
        my_filter.predict()
        my_filter.update(k)
        # do something with the output
        x = my_filter.x
        a.extend(x[0])
        b.append(k)
    #dataset.loc[dataset.num == num, dataset.columns[i]] = a
    sum_c=sum_c+a
    train_x.loc[:,'kf_X_'+str(i)]=sum_c
train_x

/tmp/ipykernel_33403/2090849507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x.drop(["X_10","X_11"], axis =1 ,inplace =True)
  0%|          | 0/54 [00:00<?, ?it/s]/tmp/ipykernel_33403/2090849507.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x.loc[:,'kf_X_'+str(i)]=sum_c
  2%|▏         | 1/54 [00:01<01:16,  1.44s/it]/tmp/ipykernel_33403/2090849507.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_12,...,kf_X_44,kf_X_45,kf_X_46,kf_X_47,kf_X_48,kf_X_49,kf_X_50,kf_X_51,kf_X_52,kf_X_53
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,4.34,...,1.002494,1.002494,9681.830424,136.706824,135.026652,147.474282,133.983516,125.297184,136.385461,124.721452
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,4.38,...,0.995193,0.995193,10463.125216,134.417826,136.614816,150.605471,124.370038,128.465304,143.906278,125.470122
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,4.36,...,0.996717,0.996717,11008.959730,132.863430,132.432688,148.165211,126.714975,127.943775,142.420532,123.066148
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,4.33,...,0.997523,0.997523,14001.464930,133.319729,132.551377,142.332612,129.976112,130.279945,146.807645,131.078737
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,4.35,...,0.998013,0.998013,12899.355654,138.704302,134.670775,136.901889,132.935585,128.206028,143.653289,127.597410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,4.36,...,1.000000,1.000000,56282.257274,129.604506,125.719101,137.318833,124.581296,125.063380,137.602621,128.758425
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,4.40,...,1.000000,1.000000,58640.674456,129.265428,124.683345,138.073689,124.194959,124.566170,138.298172,127.727242
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,4.38,...,1.000000,1.000000,52616.608456,129.711401,126.340585,137.494041,124.875757,125.353904,138.545457,126.227423
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,4.33,...,1.000000,1.000000,55226.707986,129.456898,125.494027,138.027848,125.665171,125.255583,138.178005,125.697982


In [47]:
temp_1 = ['X_14','X_15','X_16','X_17','X_18']
temp_2 = ['X_19','X_20','X_21','X_22']
temp_3 = ['X_30','X_31','X_32','X_33']

# 0값이 있는 피쳐 제외하고 log 변환 (log1p 사용 안함)
for col in basic_col:
    if col in ['X_38','X_39',"X_40","X_45"]:
        continue
    train_x[col+"_log"] = train_x[col].apply(lambda x : math.log(x))

# 가설 1. 안테나 패드 위치 간의 차이가 공정예측에 영향을 미칠 것임
for a,b in itertools.combinations(temp_1,2):
    train_x['dif_'+a+b] = train_x[a]-train_x[b]
    train_x['dif_'+b+a] = train_x[b]-train_x[a]
    
# 가설 2. 스크류 삽입 깊이의 차이가 공정예측에 영향을 미칠 것임 
for a,b in itertools.combinations(temp_2,2):
    train_x['dif_'+a+b] = train_x[a]-train_x[b]
    train_x['dif_'+b+a] = train_x[b]-train_x[a]
 
 # 가설 3. 스크류 삽입 깊이의 차이가 공정예측에 영향을 미칠 것임 
for a,b in itertools.combinations(temp_3,2):
    train_x['dif_'+a+b] = train_x[a]-train_x[b]
    train_x['dif_'+b+a] = train_x[b]-train_x[a]
    
# 가설 4. PCB 체결시 단계별 누름량 1,2의 총합이 공정예측에 영향을 미칠 것임
train_x['PCB_sum_1'] = train_x['X_01']+train_x['X_02']

# 가설 5. PCB 체결시 단계별 누름량 3,4의 총합이 공정예측에 영향을 미칠 것임
train_x['PCB_sum_2'] = train_x['X_05']+train_x['X_06']

# 가설 6. 방열재료의 총면적이 공정예측에 영향을 미칠 것임
train_x['radi_sum_area'] = train_x['X_07'] + train_x['X_08']+ train_x['X_09']

# 가설 7. 방열재료 면적당 무게가 공정예측에 영향을 미칠 것임
train_x['radi_1_weight_per_area'] = train_x['X_03'] / train_x['X_07']

# 가설 8. 레이돔 치수와 안테나 위치의 거리가 공정예측에 영향을 미칠 것임
train_x['abs_X_41X_14'] =abs(train_x["X_41"]- train_x['X_14'])
train_x['abs_X_42X_15'] =abs(train_x["X_42"]- train_x['X_15'])
train_x['abs_X_43X_16'] =abs(train_x["X_43"]- train_x['X_16'])
train_x['abs_X_44X_17'] =abs(train_x["X_44"]- train_x['X_17'])

# 가설 9. 안테나패드 위치, 스크류삽입깊이, 커넥터 핀 치수, 스크류삽입깊이의 편차(분산 또는 표준편차)가 공정예측에 영향을 미칠 것임
train_x['var_antena_loc'] = train_x[['X_14','X_15','X_16','X_17',"X_18"]].apply(lambda x : np.var(x), axis =1)
train_x['std_nthscrew_depth'] = train_x[['X_19','X_20','X_21','X_22']].apply(lambda x : np.std(x), axis =1)
train_x['std_connector'] = train_x[['X_24','X_25','X_26','X_27',"X_28","X_29"]].apply(lambda x : np.std(x), axis =1)
train_x['std_screw_depth'] = train_x[['X_30','X_31','X_32','X_33']].apply(lambda x : np.std(x), axis =1)

/tmp/ipykernel_33403/1330450477.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_x[col+"_log"] = train_x[col].apply(lambda x : math.log(x))
/tmp/ipykernel_33403/1330450477.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_x[col+"_log"] = train_x[col].apply(lambda x : math.log(x))
/tmp/ipykernel_33403/1330450477.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [48]:
train_x.to_csv('./data/train_x_no_mean_median.csv', index=False)

# test 칼만 + 각종 피쳐 생성

In [49]:
submit_x = pd.read_csv('./data/test.csv')
submit_x.drop(["ID","X_10","X_11"],axis =1 ,inplace =True)
submit_df = pd.read_csv('./data/sample_submission.csv')

basic_col = submit_x.columns


from tqdm import tqdm

for i in tqdm(range(len(submit_x.columns))):
    current=0
    sum_c=[]
    z = submit_x.loc[:, submit_x.columns[i]]
    a = []           #필터링 된 피쳐(after)
    b = []           #필터링 전 피쳐(before)
    my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
    my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
    my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
    my_filter.H = np.array([[1.,0.]])    # Measurement function
    my_filter.P *= 1000.                 # covariance matrix
    my_filter.R = 5                      # state uncertainty
    my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.1) # process uncertainty   
    for k in z.values:
        my_filter.predict()
        my_filter.update(k)
        # do something with the output
        x = my_filter.x
        a.extend(x[0])
        b.append(k)
    #dataset.loc[dataset.num == num, dataset.columns[i]] = a
    sum_c=sum_c+a
    submit_x.loc[:,'kf_X_'+str(i)]=sum_c


for col in basic_col:
    if col in ['X_38','X_39',"X_40","X_45"]:
        continue
    submit_x[col+"_log"] = submit_x[col].apply(lambda x : math.log(x))


for a,b in itertools.combinations(temp_1,2):
  submit_x['dif_'+a+b] = submit_x[a]-submit_x[b]
  submit_x['dif_'+b+a] = submit_x[b]-submit_x[a]

for a,b in itertools.combinations(temp_2,2):
  submit_x['dif_'+a+b] = submit_x[a]-submit_x[b]
  submit_x['dif_'+b+a] = submit_x[b]-submit_x[a]


for a,b in itertools.combinations(temp_3,2):
  submit_x['dif_'+a+b] = submit_x[a]-submit_x[b]
  submit_x['dif_'+b+a] = submit_x[b]-submit_x[a]

submit_x['PCB_sum_1'] = submit_x['X_01']+submit_x['X_02']
submit_x['PCB_sum_2'] = submit_x['X_05']+submit_x['X_06']
submit_x['radi_sum_area'] = submit_x['X_07'] + submit_x['X_08']+ submit_x['X_09']
submit_x['radi_1_weight_per_area'] = submit_x['X_03'] / submit_x['X_07']

submit_x['abs_X_41X_14'] =abs(submit_x["X_41"]- submit_x['X_14'])
submit_x['abs_X_42X_15'] =abs(submit_x["X_42"]- submit_x['X_15'])
submit_x['abs_X_43X_16'] =abs(submit_x["X_43"]- submit_x['X_16'])
submit_x['abs_X_44X_17'] =abs(submit_x["X_44"]- submit_x['X_17'])


submit_x['var_antena_loc'] = submit_x[['X_14','X_15','X_16','X_17',"X_18"]].apply(lambda x : np.var(x), axis =1)
submit_x['std_nthscrew_depth'] = submit_x[['X_19','X_20','X_21','X_22']].apply(lambda x : np.std(x), axis =1)
submit_x['std_connector'] = submit_x[['X_24','X_25','X_26','X_27',"X_28","X_29"]].apply(lambda x : np.std(x), axis =1)
submit_x['std_screw_depth'] = submit_x[['X_30','X_31','X_32','X_33']].apply(lambda x : np.std(x), axis =1)

100%|██████████| 54/54 [01:17<00:00,  1.44s/it]
/tmp/ipykernel_33403/4171542990.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submit_x[col+"_log"] = submit_x[col].apply(lambda x : math.log(x))
/tmp/ipykernel_33403/4171542990.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submit_x[col+"_log"] = submit_x[col].apply(lambda x : math.log(x))
/tmp/ipykernel_33403/4171542990.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [50]:
submit_x.to_csv('./data/test_x_no_mean_median.csv', index=False)

# 평균, 중간값 생성

In [51]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# import matplotlib.pyplot as plt
import time
class CFG:
    seed = 42
    fold_num = 10

    trainPath = './data/train_x_no_mean_median.csv'
    testPath = './data/test_x_no_mean_median.csv'
    
    
train_df = pd.read_csv(CFG.trainPath)
test_df = pd.read_csv(CFG.testPath)


train_new_df = train_df.copy()
test_new_df = test_df.copy()

def make_mean_median(column_list, set_amount):
    mean_arr = []
    median_arr = []
    
    # 첫 set_amount까지는 본래값 사용
    for i in range(set_amount):
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
    
    # set_amount만큼의 mean, median 생성
    for i in range(set_amount, len(column_list)):
        mean_arr.append(float(np.mean(column_list[i-set_amount:i])))
        median_arr.append(float(np.median(column_list[i-set_amount:i])))
        
    print(f'mean_arr:{len(mean_arr)}')
    print(f'median_arr:{len(median_arr)}')
    return mean_arr.copy(), median_arr.copy()

def make_move_avg_median(df, save_file_name, type=None, set_amount=30):
    # set_amount 조절가능
    # trainX -> X_10, X_11을 제외한 54개의 이동평균
    pbar = tqdm(df.columns[:54])
    for column in pbar:
        column_list = df[column].to_list()
        pbar.set_description(f"The mean and median of {column} is making")
        meanArr, medianArr = make_mean_median(column_list, set_amount)
        
        df[f'{column}_mean_{set_amount}'] = meanArr
        df[f'{column}_median_{set_amount}'] = medianArr
        
    df.to_csv(save_file_name, index=False)


make_move_avg_median(train_df, save_file_name='./data/train_x_engineered.csv', type='train')
make_move_avg_median(test_df, save_file_name='./data/test_x_engineered.csv', type='test')

The mean and median of X_02 is making:   2%|▏         | 1/54 [00:01<01:03,  1.19s/it]

mean_arr:39607
median_arr:39607


The mean and median of X_03 is making:   4%|▎         | 2/54 [00:02<01:01,  1.19s/it]

mean_arr:39607
median_arr:39607


The mean and median of X_04 is making:   6%|▌         | 3/54 [00:03<01:00,  1.18s/it]

...

mean_arr:39608
median_arr:39608


/tmp/ipykernel_33403/1104968906.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_mean_{set_amount}'] = meanArr
/tmp/ipykernel_33403/1104968906.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_median_{set_amount}'] = medianArr
The mean and median of X_56 is making:  98%|█████████▊| 53/54 [01:02<00:01,  1.19s/it]

mean_arr:39608
median_arr:39608


/tmp/ipykernel_33403/1104968906.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_mean_{set_amount}'] = meanArr
/tmp/ipykernel_33403/1104968906.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_median_{set_amount}'] = medianArr
The mean and median of X_56 is making: 100%|██████████| 54/54 [01:03<00:00,  1.18s/it]

mean_arr:39608
median_arr:39608
